In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [6]:
import chardet

Below is my solution to a UnicodeDecodeError problem I was having. I could have done the following other things, which I will keep here as personal notes:

1.
with open('filename.csv', 'rb') as f: result = chardet.detect(f.read()) # or readline if the file is large such as f.read(10242) for the first megabyte.
pd.read_csv('filename.csv', encoding=result['encoding'])

2.
df = pd.read_csv(r"filename.csv", encoding="cp1252") # r is for "raw string" to further avoid complications

3.
Opening the csv in notepad++, going to encoding menu, choose utf-8, and save over file OR go to sublimetext and save with utf-8 encoding THEN load the overwritten csv (will have to download either of these to do this option).

4.
Another option is to cover up the errors by passing an error handler:

ifd = open("infile.csv", 'r', errors='replace')

5. 
My Mentor's Method (didn't work for me)
import csv

with open('../Data/Welcome_Trust_Prices_2012_to_2013/WELLCOME_APCspend2013_forThinkful.csv',
          newline='', encoding='utf-8') as file:
    for row in csv.reader(file):
        print(row)

__NOTE:__ I did not test to see if any of these options actually work (although the third option seems that it would work intuitively). Again, just noting for posterity.

In [310]:
df = pd.read_csv('../Data/Welcome_Trust_Prices_2012_to_2013/WELLCOME_APCspend2013_forThinkful.csv', encoding='latin-1')
### setting the encoding to utf-8 still pushed a decode error. I can play around with the original file to find the fault
# but using latin 1 is a quick workaround, and I haven't seen any resulting mojibake.

df.tail()

,PMID/PMCID,Publisher,Journal title,Article title,COST (£) charged to Wellcome (inc VAT when charged)
2122,2901593,Wolters Kluwer Health,Circulation Research,Mechanistic Links Between Na+ Channel (SCN5A) ...,£1334.15
2123,3748854,Wolters Kluwer Health,AIDS,Evaluation of an empiric risk screening score ...,£1834.77
2124,3785148,Wolters Kluwer Health,Pediatr Infect Dis J,Topical umbilical cord care for prevention of ...,£1834.77
2125,PMCID:\n PMC3647051\n,Wolters Kluwer N.V./Lippinott,AIDS,Grassroots Community Organisations' Contributi...,£2374.52
2126,PMID: 23846567 (Epub July 2013),Wolters Kluwers,Journal of Acquired Immune Deficiency Syndromes,A novel community health worker tool outperfor...,£2034.75


## Assignment Goals

1. Determine the five most common journals and the total articles for each. 

2. Next, calculate the mean, median, and standard deviation of the open-access cost per article for each journal. You will need to do considerable data cleaning in order to extract accurate estimates, and may want to look into data encoding methods if you get stuck. 

3. For a real bonus round, identify the open access prices paid by subject area.

In [311]:
# for the first goal
df['Journal title'].value_counts() # I see the Plos One problem...

PLoS One                                                              92
PLoS ONE                                                              62
Journal of Biological Chemistry                                       48
Nucleic Acids Research                                                21
Proceedings of the National Academy of Sciences                       19
PLoS Neglected Tropical Diseases                                      18
Human Molecular Genetics                                              18
Nature Communications                                                 17
PLoS Genetics                                                         15
PLoS Pathogens                                                        15
Neuroimage                                                            15
BMC Public Health                                                     14
Brain                                                                 14
NeuroImage                                         

In [312]:
# making various replacements

df['Journal title'] = df['Journal title'].str.replace('J ', 'Journal ')
df['Journal title'] = df['Journal title'].str.replace('fro ', 'for ')
df['Journal title'] = df['Journal title'].str.replace('The ', '')
df['Journal title'] = df['Journal title'].str.replace('the ', '')

df.head()

,PMID/PMCID,Publisher,Journal title,Article title,COST (£) charged to Wellcome (inc VAT when charged)
0,NaN,CUP,Psychological Medicine,Reduced parahippocampal cortical thickness in ...,£0.00
1,PMC3679557,ACS,Biomacromolecules,Structural characterization of a Model Gram-ne...,£2381.04
2,23043264 PMC3506128,ACS,Journal Med Chem,"Fumaroylamino-4,5-epoxymorphinans and related ...",£642.56
3,23438330 PMC3646402,ACS,Journal Med Chem,Orvinols with mixed kappa/mu opioid receptor a...,£669.64
4,23438216 PMC3601604,ACS,Journal Org Chem,Regioselective opening of myo-inositol orthoes...,£685.88


In [313]:
# doing it piecemeal just in case something breaks 

df['Journal title'] = df['Journal title'].str.replace('Biol ', 'Biological ')
df['Journal title'] = df['Journal title'].str.replace('Med ', 'Medical ')
df['Journal title'] = df['Journal title'].str.replace('Chem ', ' Chemistry ')
df['Journal title'] = df['Journal title'].str.replace('BioMed Central', 'BMC')

In [314]:
df['Journal title'].replace('Pediatr Infect Dis J', 'Pediatric Infection Dis Journal', inplace=True)
df.tail()

# realized I had to do "inplace" for correct replacement.
# why else would I do a replacement if not for inplace?!?!?!

,PMID/PMCID,Publisher,Journal title,Article title,COST (£) charged to Wellcome (inc VAT when charged)
2122,2901593,Wolters Kluwer Health,Circulation Research,Mechanistic Links Between Na+ Channel (SCN5A) ...,£1334.15
2123,3748854,Wolters Kluwer Health,AIDS,Evaluation of an empiric risk screening score ...,£1834.77
2124,3785148,Wolters Kluwer Health,Pediatric Infection Dis Journal,Topical umbilical cord care for prevention of ...,£1834.77
2125,PMCID:\n PMC3647051\n,Wolters Kluwer N.V./Lippinott,AIDS,Grassroots Community Organisations' Contributi...,£2374.52
2126,PMID: 23846567 (Epub July 2013),Wolters Kluwers,Journal of Acquired Immune Deficiency Syndromes,A novel community health worker tool outperfor...,£2034.75


In [315]:
df['Journal title'] = df['Journal title'].str.replace('Jnl ', 'Journal ')
df['Journal title'] = df['Journal title'].str.replace('British Medical Journal', 'BMJ')
df['Journal title'] = df['Journal title'].str.replace('&', 'and')
df['Journal title'] = df['Journal title'].str.replace('Virol ', 'Virology')
df['Journal title'] = df['Journal title'].str.replace('Immunol ', 'Immunology')
df['Journal title'] = df['Journal title'].str.replace('Mol ', 'Molecular ') 

            
df.head()

,PMID/PMCID,Publisher,Journal title,Article title,COST (£) charged to Wellcome (inc VAT when charged)
0,NaN,CUP,Psychological Medicine,Reduced parahippocampal cortical thickness in ...,£0.00
1,PMC3679557,ACS,Biomacromolecules,Structural characterization of a Model Gram-ne...,£2381.04
2,23043264 PMC3506128,ACS,Journal Medical Chem,"Fumaroylamino-4,5-epoxymorphinans and related ...",£642.56
3,23438330 PMC3646402,ACS,Journal Medical Chem,Orvinols with mixed kappa/mu opioid receptor a...,£669.64
4,23438216 PMC3601604,ACS,Journal Org Chem,Regioselective opening of myo-inositol orthoes...,£685.88


In [316]:

df['Journal title'] = df['Journal title'].str.replace('Infect ', 'Infection ')
df['Journal title'] = df['Journal title'].str.replace('Biosci', 'Bioscience')
df['Journal title'] = df['Journal title'].str.replace('Org ', 'Organic ')
df['Journal title'] = df['Journal title'].str.replace('(', '') #taking out punctuation
df['Journal title'] = df['Journal title'].str.replace(')', '')
df['Journal title'] = df['Journal title'].str.replace('.', '')
df['Journal title'] = df['Journal title'].str.replace(',', '')

df.head()

,PMID/PMCID,Publisher,Journal title,Article title,COST (£) charged to Wellcome (inc VAT when charged)
0,NaN,CUP,Psychological Medicine,Reduced parahippocampal cortical thickness in ...,£0.00
1,PMC3679557,ACS,Biomacromolecules,Structural characterization of a Model Gram-ne...,£2381.04
2,23043264 PMC3506128,ACS,Journal Medical Chem,"Fumaroylamino-4,5-epoxymorphinans and related ...",£642.56
3,23438330 PMC3646402,ACS,Journal Medical Chem,Orvinols with mixed kappa/mu opioid receptor a...,£669.64
4,23438216 PMC3601604,ACS,Journal Organic Chem,Regioselective opening of myo-inositol orthoes...,£685.88


In [317]:
# Now to avoid all capitalization problems

df['Journal'] = df['Journal title'].str.upper() 

# I should've done this in the beginning before I made my replacements
# but the code was already typed out, and I've already made enough tedious changes.

df.head(4)

,PMID/PMCID,Publisher,Journal title,Article title,COST (£) charged to Wellcome (inc VAT when charged),Journal
0,NaN,CUP,Psychological Medicine,Reduced parahippocampal cortical thickness in ...,£0.00,PSYCHOLOGICAL MEDICINE
1,PMC3679557,ACS,Biomacromolecules,Structural characterization of a Model Gram-ne...,£2381.04,BIOMACROMOLECULES
2,23043264 PMC3506128,ACS,Journal Medical Chem,"Fumaroylamino-4,5-epoxymorphinans and related ...",£642.56,JOURNAL MEDICAL CHEM
3,23438330 PMC3646402,ACS,Journal Medical Chem,Orvinols with mixed kappa/mu opioid receptor a...,£669.64,JOURNAL MEDICAL CHEM


In [277]:
df2 = df
df2['Publisher'].replace('CUP', 'dog', inplace=True)
df2['Publisher'] = df2['Publisher'].replace('dog', 'ACS')
df2.head()

,PMID/PMCID,Publisher,Journal title,Article title,COST (£) charged to Wellcome (inc VAT when charged),Journal
0,NaN,ACS,Psychological Medicine,Reduced parahippocampal cortical thickness in ...,£0.00,PSYCHOLOGICAL MEDICINE
1,PMC3679557,ACS,Biomacromolecules,Structural characterization of a Model Gram-ne...,£2381.04,BIOMACROMOLECULES
2,23043264 PMC3506128,ACS,J Med Chem,"Fumaroylamino-4,5-epoxymorphinans and related ...",£642.56,J MED CHEM
3,23438330 PMC3646402,ACS,J Med Chem,Orvinols with mixed kappa/mu opioid receptor a...,£669.64,J MED CHEM
4,23438216 PMC3601604,ACS,J Org Chem,Regioselective opening of myo-inositol orthoes...,£685.88,J ORG CHEM


In [318]:
df.Journal.value_counts()

PLOS ONE                                                           190
JOURNAL OF BIOLOGICAL CHEMISTRY                                     60
NEUROIMAGE                                                          29
PLOS PATHOGENS                                                      24
PLOS GENETICS                                                       24
NUCLEIC ACIDS RESEARCH                                              23
PLOS NEGLECTED TROPICAL DISEASES                                    20
PROCEEDINGS OF NATIONAL ACADEMY OF SCIENCES                         20
NATURE COMMUNICATIONS                                               19
HUMAN MOLECULAR GENETICS                                            19
BIOCHEMICAL JOURNAL                                                 14
BMC PUBLIC HEALTH                                                   14
BRAIN                                                               14
JOURNAL OF NEUROSCIENCE                                             14
MOVEME

## Answer to Question 1: The top five most common Journals are:
1. PLoS One (190 articles)
2. Journal of Biological Chemistry (60 articles)
3. Neuroimage (29 articles)
4. PLoS Pathogens (24 articles)
5. PLoS Genetics (24 articles)

In [319]:
# there can still be more cleaning to do, but I think it's good for now...


# for second goal: Next, calculate the mean, median, and stan. dev of the cost per article for EACH JOURNAL.
df.rename(columns={'COST (£) charged to Wellcome (inc VAT when charged)':'Cost'},inplace=True)

In [320]:
df.head()

,PMID/PMCID,Publisher,Journal title,Article title,Cost,Journal
0,NaN,CUP,Psychological Medicine,Reduced parahippocampal cortical thickness in ...,£0.00,PSYCHOLOGICAL MEDICINE
1,PMC3679557,ACS,Biomacromolecules,Structural characterization of a Model Gram-ne...,£2381.04,BIOMACROMOLECULES
2,23043264 PMC3506128,ACS,Journal Medical Chem,"Fumaroylamino-4,5-epoxymorphinans and related ...",£642.56,JOURNAL MEDICAL CHEM
3,23438330 PMC3646402,ACS,Journal Medical Chem,Orvinols with mixed kappa/mu opioid receptor a...,£669.64,JOURNAL MEDICAL CHEM
4,23438216 PMC3601604,ACS,Journal Organic Chem,Regioselective opening of myo-inositol orthoes...,£685.88,JOURNAL ORGANIC CHEM


In [343]:
dfFixedCost = []
for value in df.Cost:
    newvalue = value.replace("£", "") # The split method proposed by Thinkful works too!
    dfFixedCost.append(newvalue)
    
df['FixedCost'] = dfFixedCost
df.head()
    

,PMID/PMCID,Publisher,Journal title,Article title,Cost,Journal,FixedCost
0,NaN,CUP,Psychological Medicine,Reduced parahippocampal cortical thickness in ...,£0.00,PSYCHOLOGICAL MEDICINE,0.00
1,PMC3679557,ACS,Biomacromolecules,Structural characterization of a Model Gram-ne...,£2381.04,BIOMACROMOLECULES,2381.04
2,23043264 PMC3506128,ACS,Journal Medical Chem,"Fumaroylamino-4,5-epoxymorphinans and related ...",£642.56,JOURNAL MEDICAL CHEM,642.56
3,23438330 PMC3646402,ACS,Journal Medical Chem,Orvinols with mixed kappa/mu opioid receptor a...,£669.64,JOURNAL MEDICAL CHEM,669.64
4,23438216 PMC3601604,ACS,Journal Organic Chem,Regioselective opening of myo-inositol orthoes...,£685.88,JOURNAL ORGANIC CHEM,685.88


In [344]:
len(df)

2127

In [345]:
dfFixedCost[485:495]

['2539.89',
 '2515.57',
 '1366.08',
 '3208.69',
 '3948.42',
 '4163.88',
 '999999.00',
 '2363.14',
 '3838.88',
 '4041.05']

In [346]:
# So the above cell shows two things:
# (1) There are broken values that need fixing
# (2) The quotes show that these numbers are strings, confirmed below...
print(dfFixedCost[1])
print(dfFixedCost[2])
print(dfFixedCost[1] + dfFixedCost[2])
type(dfFixedCost[1])

2381.04
642.56
2381.04642.56


str

In [349]:
dfFixedCost = df['FixedCost'] # I screwed up the list with a lambda function - had to reset the values
print(type(dfFixedCost[1])) # just to triple check the object type

dfFixedCosttemp = [] # I tried all other simpler ways of converting to float...

for value in dfFixedCost:
    value = ''.join(value.split('$'))
    value = float(value)
    dfFixedCosttemp.append(value)  # required to overwrite and map accordingly

dfFixedCost = dfFixedCosttemp
dfFixedCost[1] + dfFixedCost[2]

<class 'str'>


3023.6

In [350]:
# okay, now that all string values are FINALLY converted to float...
# time to fix erroneous outlier values
dfFixedCostnorm = [x for x in dfFixedCost if x < 10000]
print(np.median(dfFixedCostnorm))
print(np.mean(dfFixedCostnorm))

1851.29
1822.0559075589792


In [351]:
# going to use median...
# list comprehension syntax
# [x for x in LIST if x CONDITION]
# OR USE: [x if CONDITION else CONDITION2 for x in LIST]


dfFixedCost = [x if x < 10000 else 1851.29 for x in dfFixedCost] # again, the median for a list of all non-outlier values
dfFixedCost[485:495]
        

[2539.89,
 2515.57,
 1366.08,
 3208.69,
 3948.42,
 4163.88,
 1851.29,
 2363.14,
 3838.88,
 4041.05]

In [352]:
# plugging back into df
df['FixedCost'] = dfFixedCost
df.head()

,PMID/PMCID,Publisher,Journal title,Article title,Cost,Journal,FixedCost
0,NaN,CUP,Psychological Medicine,Reduced parahippocampal cortical thickness in ...,£0.00,PSYCHOLOGICAL MEDICINE,0.00
1,PMC3679557,ACS,Biomacromolecules,Structural characterization of a Model Gram-ne...,£2381.04,BIOMACROMOLECULES,2381.04
2,23043264 PMC3506128,ACS,Journal Medical Chem,"Fumaroylamino-4,5-epoxymorphinans and related ...",£642.56,JOURNAL MEDICAL CHEM,642.56
3,23438330 PMC3646402,ACS,Journal Medical Chem,Orvinols with mixed kappa/mu opioid receptor a...,£669.64,JOURNAL MEDICAL CHEM,669.64
4,23438216 PMC3601604,ACS,Journal Organic Chem,Regioselective opening of myo-inositol orthoes...,£685.88,JOURNAL ORGANIC CHEM,685.88


In [353]:
print("The mean of the open-access cost per article is ${} +/- ${}.".format(np.mean(df.FixedCost), np.std(df.FixedCost)))
print("The median cost of the open-access cost per article is ${}.".format(np.median(df.FixedCost)))


The mean of the open-access cost per article is $1822.743121767748 +/- $758.4519244115404.
The median cost of the open-access cost per article is $1851.29.


In [354]:
# determining central tendencies for * each journal *, as requested by question 2:

# now for the groupby grand finale!!!!
dftest = df # made this new df as to avoid screwing up original df
dftest1 = df.groupby(by='Journal').FixedCost.std().reset_index()
dftest1.columns = ('Journal', 'Stan. Dev.') # had to rename the std numerical column
dftest2 = df.groupby(by='Journal').FixedCost.mean().reset_index()
dftest2.columns = ('Journal', 'Average Cost')
dftest3 = df.groupby(by='Journal').FixedCost.median().reset_index()
dftest3.columns = ('Journal', 'Median Cost')

# pd.concat didn't work - it overwrote the numerical columns - did a manual pd.merge
dftestfinal = pd.merge(dftest3, dftest2)
dftestfinal = pd.merge(dftestfinal, dftest1)
dftestfinal = dftestfinal.sort_values(by=['Median Cost'], ascending=False)
dftestfinal.head()


,Journal,Median Cost,Average Cost,Stan. Dev.
798,PUBLIC SERVICE REVIEW,6000.000,6000.0000,NaN
618,LANCET NEUROLOGY,5040.000,5040.0000,1018.233765
614,LANCET,4677.005,4618.5025,230.322752
242,CURR BIOL,4151.770,4151.7700,NaN
193,CELL JOURNAL,4041.050,4041.0500,NaN


In [355]:
# fixing NaN's
dftestfinal['Stan. Dev.'].fillna(0, inplace=True)
#  had to do this^^^ instead of list comprehension because 'NaN' is not a string, and not a number (obviously)

# ANSWER TO QUESTION 2:
dftestfinal.head()

,Journal,Median Cost,Average Cost,Stan. Dev.
798,PUBLIC SERVICE REVIEW,6000.000,6000.0000,0.000000
618,LANCET NEUROLOGY,5040.000,5040.0000,1018.233765
614,LANCET,4677.005,4618.5025,230.322752
242,CURR BIOL,4151.770,4151.7700,0.000000
193,CELL JOURNAL,4041.050,4041.0500,0.000000


### For third goal: identify the open access prices paid by subject area.

For open-access prices, groupby identify the open access prices paid by subject area.
HOW DO I DO THIS? What is meant by "subject area?" there is no column with such information.


In [184]:
dftestfinal.head()

,Journal,Median Cost,Average Cost,Stan. Dev.
812,PUBLIC SERVICE REVIEW,6000.00,6000.000000,0.000000
891,THE LANCET NEUROLOGY,5040.00,5040.000000,1018.233765
627,LANCET,4800.00,4800.000000,0.000000
890,THE LANCET,4554.01,4558.003333,240.024915
196,CELL HOST & MICROBE,4226.04,4226.040000,0.000000


In [456]:
# creating keyword lists

genrebio = set(['BIOLOGY', 'BIOLOGICAL', 'BIOL', 'CELL'])
genrechem = set(['CHEMISTRY', 'CHEMICAL', 'BIOCHEMISTRY', 'BIOCHEM'])
genremicro = set(['MICROBIOLOGY', 'VIRULOLOGY', 'MICROBIOL', 'IMMUNOLOGY'])
genrephysics = set(['PHYSICS', 'PHYSICAL'])
genregenetics = set(['GENETICS', 'GENETIC'])
genreneurology = set(['NEUROLOGY', 'NEURO', 'NEUROSCIENCE'])




In [332]:
dfgenre = dftestfinal # making new df in case I screw up the data

In [447]:
dfgenre['Subject Area'] = dfgenre.Journal
dfgenre.head(7)

,Journal,Median Cost,Average Cost,Stan. Dev.,Subject Area
798,PUBLIC SERVICE REVIEW,6000.000,6000.0000,0.000000,PUBLIC SERVICE REVIEW
618,LANCET NEUROLOGY,5040.000,5040.0000,1018.233765,LANCET NEUROLOGY
614,LANCET,4677.005,4618.5025,230.322752,LANCET
242,CURR BIOL,4151.770,4151.7700,0.000000,CURR BIOL
193,CELL JOURNAL,4041.050,4041.0500,0.000000,CELL JOURNAL
192,CELL HOST AND MICROBE,4032.460,4032.4600,273.763461,CELL HOST AND MICROBE
417,IMMUNITY,3934.750,3934.7500,190.791552,IMMUNITY


In [466]:
# Careful of running this cell! 
# It's very process-heavy!!!!! Use judiciously!

for i in range(dfgenre.shape[0]):
    tokens = set(dfgenre.Journal.iloc[i].split())
    if len(tokens.intersection(genreneurology)) > 0:
        dfgenre['Subject Area'].iloc[i] = 'Neurology'
        
    elif len(tokens.intersection(genrebio)) > 0:
        dfgenre['Subject Area'].iloc[i] = 'Biology'
        
    elif len(tokens.intersection(genrechem)) > 0:
        dfgenre['Subject Area'].iloc[i] = 'Chemistry'    
    
    elif len(tokens.intersection(genremicro)) > 0:
        dfgenre['Subject Area'].iloc[i] = 'Microbiology'
    
    elif len(tokens.intersection(genrephysics)) > 0:
        dfgenre['Subject Area'].iloc[i] = 'Physics'
    
    elif len(tokens.intersection(genregenetics)) > 0:
        dfgenre['Subject Area'].iloc[i] = 'Genetics'
        
    else:
        dfgenre['Subject Area'].iloc[i] = 'Other' 
        # ^^^^This is why I put it in a single loop instead of breaking it up:
        # I don't want to overwrite everything with "Other", 
        # which would happen if I broke it up into multiple loops
    

In [467]:
dfgenre.head()

,Journal,Median Cost,Average Cost,Stan. Dev.,Subject Area
798,PUBLIC SERVICE REVIEW,6000.000,6000.0000,0.000000,Other
618,LANCET NEUROLOGY,5040.000,5040.0000,1018.233765,Neurology
614,LANCET,4677.005,4618.5025,230.322752,Other
242,CURR BIOL,4151.770,4151.7700,0.000000,Biology
193,CELL JOURNAL,4041.050,4041.0500,0.000000,Biology


In [468]:
# Answer to question 3:
dfgenre

,Journal,Median Cost,Average Cost,Stan. Dev.,Subject Area
798,PUBLIC SERVICE REVIEW,6000.000,6000.000000,0.000000,Other
618,LANCET NEUROLOGY,5040.000,5040.000000,1018.233765,Neurology
614,LANCET,4677.005,4618.502500,230.322752,Other
242,CURR BIOL,4151.770,4151.770000,0.000000,Biology
193,CELL JOURNAL,4041.050,4041.050000,0.000000,Biology
192,CELL HOST AND MICROBE,4032.460,4032.460000,273.763461,Biology
417,IMMUNITY,3934.750,3934.750000,190.791552,Other
695,NEURON,3918.420,3565.644444,1116.765911,Other
194,CELL METABOLISM,3904.460,3924.260000,151.324659,Biology
847,STRUCTURE,3895.640,3881.824286,166.835165,Other
